In [87]:
# FIX Vocabulary

In [88]:
import sys
from gastrodon import *
from rdflib import *
import pandas as pd
pd.options.display.width=120
pd.options.display.max_colwidth=100

In [89]:
g=Graph()

In [90]:
### Load local FIX Vocabulary file in turtle format into a graph provided RDFLib.

In [91]:
g.parse("fix-vocabulary.ttl",format="turtle")

<Graph identifier=Ne60fb535eb6f4508a9c9b535db43b474 (<class 'rdflib.graph.Graph'>)>

In [92]:
len(g)

1564

In [93]:
g.bind("owl","http://www.w3.org/2002/07/owl#")
g.bind("dct", "http://purl.org/dc/terms/")
g.bind("skos","http://www.w3.org/2004/02/skos/core#")
e=LocalEndpoint(g)

In [ ]:
### Query ontology metadata

In [94]:
s=e.select("""
   SELECT ?s {
      ?s rdf:type owl:Ontology .
    }
""")
s

,s
0,https://fixprotocol.io/vocabulary


In [95]:
ontology=one(s)
ontology

rdflib.term.URIRef('https://fixprotocol.io/vocabulary')

In [96]:
meta=e.select("""
    SELECT ?p ?o 
    WHERE {
        ?_ontology ?p ?o .
    } 
""")
meta

,p,o
0,rdf:type,owl:Ontology
1,dct:rights,Copyright 2018 FIX Trading Community
2,dct:date,2018-06-04 18:40:35.584612+00:00
3,dct:license,Apache License Version 2.0
4,dct:title,FIX Vocabulary


In [97]:
### Query all terms

In [103]:
terms=e.select("""
    SELECT ?term ?definition ?from 
    WHERE {
        ?concept  skos:prefLabel ?term . 
        ?concept skos:definition ?definition .
        OPTIONAL { ?concept dct:isPartOf ?from . }
    }
""")
terms

,term,definition,from
0,Down and In Option,the option becomes active if the underlying price drops below the specified barrier.,EP92
1,Multileg-to-Multileg Trade,Identifies that a multileg order has executed against another multileg order in a \r\nmultileg o...,None
2,an issuer,Quoting and other messages\r\n\r\nVolume 7,None
3,Last Delivery Date,Specifies the last delivery date of the delivery period for a physically delivered \r\ninstrument.,None
4,FIX Session,"A FIX Session is comprised of one or more FIX Connections, meaning that a FIX \r\nSession spans ...",None
5,Off Hours Trade,A trade that occurs outside normal trading hours. Used to qualify privately negotiated \r\ntrade...,None
6,Binary Option,A binary option is a cash settled option that has a discontinuous payoff. Binary options come i...,None
7,Put Date,The date on which the buyer of a security has the right but not the obligation to sell the \r\ns...,None
8,Binary All or None Option,Fixed pay out if the underlying settles on a predefined trigger price.,EP92
9,Executing Unit,"Identifies executing unit within an Executing Firm, e.g. trading desk, branch office or",None


In [ ]:
### Query similar terms

In [104]:
options=e.select("""
    SELECT ?term ?definition 
    WHERE {
        ?concept  skos:prefLabel ?term . 
        ?concept skos:definition ?definition .
        FILTER regex(?term, "Option") 
    }
""")
options

,term,definition
0,Down and In Option,the option becomes active if the underlying price drops below the specified barrier.
1,Binary Option,A binary option is a cash settled option that has a discontinuous payoff. Binary options come i...
2,Binary All or None Option,Fixed pay out if the underlying settles on a predefined trigger price.
3,Capped Call Option,The option has a linear payout (like a vanilla) up to a capped amount as specified by a \r\ncap ...
4,Floored Put Option,The option has a linear payout (like a vanilla) up to a capped amount as specified by a \r\nfloo...
5,Average Price (Asian) Option,The underlying price is an average of the daily settlement prices over a specified \r\nperiod
6,Barrier Option,The option becomes active (knock-in) or inactive (knock-out) based on a predetermined price \r\n...
7,Binary Barrier Option,A digital option which becomes active or inactive based on the crossing of a barrier. \r\nEvents...
8,Up and Out Option,the option becomes inactive if the underlying price exceeds the specified barrier.
9,Fixed Payout Option,"The payout amount is specified at inception. Associated with Binary options (Yes, it pays \r\nor..."
